# Load JPEG-2000 files and SAFE format

In this notebook a Sentinel2-L2A saved in a SAFE format will be loaded locally into a single DataCube ready for FuseTS algorithm.

It is assumed that the user already has a SAFE product folder, however this notebook provides a method to download and extract a sample product (~900 MB) using the standard python libraries `urllib` and `zipfile`.

In [1]:
from pathlib import Path

demo_product_url = 'https://artifactory.vgt.vito.be/auxdata-public/ai4food/S2B_MSIL2A_20230905T082609_N0509_R021_T36SWD_20230905T110236.SAFE.zip'
product_name = 'S2B_MSIL2A_20230905T082609_N0509_R021_T36SWD_20230905T110236.SAFE'

destination_folder = Path('./demo_product/')

product_folder = destination_folder / product_name

In [2]:
import urllib.request
import zipfile
import io

# We download and extract the zip archive to target folder
if not product_folder.exists():
    try:
        with urllib.request.urlopen(demo_product_url) as response:
            if response.getcode() == 200:
                with zipfile.ZipFile(io.BytesIO(response.read()), 'r') as zip_ref:
                    zip_ref.extractall(destination_folder)
            else:
                print('Failed to download the zip file.')
    except Exception as e:
        print(f'Exception raised: {e}')

# Check that the files are correctly extracted
list(Path.iterdir(destination_folder))

[PosixPath('demo_product/S2B_MSIL2A_20230905T082609_N0509_R021_T36SWD_20230905T110236.SAFE')]

### Parsing the .SAFE folder

The different bands of the file are stored different subfolders, we can find the path of all those subfolders using the `manifest.SAFE` file provided in every SAFE product, which is an XML file containg various metadata information, but also the relative paths to the raster files.

In [3]:
import xml.etree.ElementTree as ET


# Parses the manifest.SAFE file for elements
tree = ET.parse(product_folder / 'manifest.safe')
root = tree.getroot()

data_objects = root.find('dataObjectSection').findall('dataObject')

band_files = {}
searched_bands = [
    'B02_10m', 'B03_10m', 'B04_10m', 'B05_20m', 'B06_20m', 'B07_20m', 'B08_10m',
    'B8A_20m', 'B09_60m', 'B11_60m', 'B12_60m', 'SCL_20m'
]

for data_object in data_objects:
    object_id = data_object.attrib['ID']  # The ID of the file
    file_location = data_object.find('byteStream').find('fileLocation').attrib['href']  # The path to that file

    # Searches for a match of the band name within the ID of the object
    for band in searched_bands:
        if band in object_id:
            band_files[band] = product_folder / file_location
    
band_files

{'B02_10m': PosixPath('demo_product/S2B_MSIL2A_20230905T082609_N0509_R021_T36SWD_20230905T110236.SAFE/GRANULE/L2A_T36SWD_A033938_20230905T083850/IMG_DATA/R10m/T36SWD_20230905T082609_B02_10m.jp2'),
 'B03_10m': PosixPath('demo_product/S2B_MSIL2A_20230905T082609_N0509_R021_T36SWD_20230905T110236.SAFE/GRANULE/L2A_T36SWD_A033938_20230905T083850/IMG_DATA/R10m/T36SWD_20230905T082609_B03_10m.jp2'),
 'B04_10m': PosixPath('demo_product/S2B_MSIL2A_20230905T082609_N0509_R021_T36SWD_20230905T110236.SAFE/GRANULE/L2A_T36SWD_A033938_20230905T083850/IMG_DATA/R10m/T36SWD_20230905T082609_B04_10m.jp2'),
 'B08_10m': PosixPath('demo_product/S2B_MSIL2A_20230905T082609_N0509_R021_T36SWD_20230905T110236.SAFE/GRANULE/L2A_T36SWD_A033938_20230905T083850/IMG_DATA/R10m/T36SWD_20230905T082609_B08_10m.jp2'),
 'B05_20m': PosixPath('demo_product/S2B_MSIL2A_20230905T082609_N0509_R021_T36SWD_20230905T110236.SAFE/GRANULE/L2A_T36SWD_A033938_20230905T083850/IMG_DATA/R20m/T36SWD_20230905T082609_B05_20m.jp2'),
 'B06_20m': Pos

### Rioxarray library

The `rioxarray` library in a `xarray` extension allowing to load raster files such as GeoTIFF and JPEG-2000 (which is the format on which SAFE files are saved).

Because xarray doesn't support those file formats by default, the extension package must be installed.

In [4]:
!pip3 install rioxarray

Looking in indexes: https://artifactory.vgt.vito.be/api/pypi/python-packages/simple


### Loading the cube.

We load the bands one by one using `rioxarray`, and concatenate them into a final cube ready to be used by FuseTS algorithms.

Because an entire sentinel2 tile can be a lot of data to handle, let's only work on a subset

In [8]:
import xarray as xr
import rioxarray

loaded_bands = []

# Fetch the common coordintes. Because B02 has a 10m resolution, it has the 
# best coordinates to match the others
arr_b02 = rioxarray.open_rasterio(
    band_files['B02_10m']
).isel(x=slice(0, 1024), y=slice(0, 1024))

common_x_coords = arr_b02.coords['x']
common_y_coords = arr_b02.coords['y']

del arr_b02

for band_name, band_path in band_files.items():
    inarr = rioxarray.open_rasterio(band_path).astype('uint16')

    inarr = inarr.interp(
        x=common_x_coords, y=common_y_coords, method='nearest', kwargs={
            'fill_value': 'extrapolate'
        }
    ).astype('uint16')
    loaded_bands.append(inarr)

In [9]:
cube = xr.concat(loaded_bands, dim='bands').assign_coords({'bands': list(band_files.keys())})
cube

<xarray.DataArray (bands: 12, band: 1, y: 1024, x: 1024)>
array([[[[2188, 2174, 2160, ..., 2526, 2536, 2488],
         [2174, 2184, 2188, ..., 2640, 2682, 2654],
         [2150, 2208, 2240, ..., 2656, 2628, 2660],
         ...,
         [2032, 2196, 2060, ..., 1709, 1734, 1733],
         [2242, 2224, 2098, ..., 2006, 1993, 1998],
         [2384, 2204, 2186, ..., 2262, 2274, 2250]]],


       [[[2490, 2476, 2480, ..., 2826, 2896, 2830],
         [2494, 2520, 2516, ..., 2954, 3078, 3084],
         [2504, 2522, 2536, ..., 3160, 3036, 3204],
         ...,
         [2470, 2666, 2528, ..., 1929, 1932, 1892],
         [2686, 2686, 2600, ..., 2352, 2358, 2274],
         [2836, 2706, 2714, ..., 2656, 2704, 2728]]],


       [[[2986, 2996, 3002, ..., 3260, 3340, 3312],
         [2990, 3008, 3046, ..., 3474, 3588, 3618],
...
         [4260, 4260, 4260, ..., 4325, 4325, 4325],
         [4260, 4260, 4260, ..., 4325, 4325, 4325]]],


       [[[5137, 5137, 5137, ..., 5677, 5677, 5677],
         [5137, 5137, 5137, ..., 5677, 5677, 5677],
         [5137, 5137, 5137, ..., 5677, 5677, 5677],
         ...,
         [5186, 5186, 5186, ..., 3661, 3661, 3661],
         [5186, 5186, 5186, ..., 3661, 3661, 3661],
         [5186, 5186, 5186, ..., 3661, 3661, 3661]]],


       [[[4455, 4455, 4455, ..., 4957, 4957, 4957],
         [4455, 4455, 4455, ..., 4957, 4957, 4957],
         [4455, 4455, 4455, ..., 4957, 4957, 4957],
         ...,
         [4498, 4498, 4498, ..., 3118, 3118, 3118],
         [4498, 4498, 4498, ..., 3118, 3118, 3118],
         [4498, 4498, 4498, ..., 3118, 3118, 3118]]]], dtype=uint16)
Coordinates:
  * band         (band) int64 1
    spatial_ref  int64 0
  * x            (x) float64 5e+05 5e+05 5e+05 ... 5.102e+05 5.102e+05 5.102e+05
  * y            (y) float64 3.9e+06 3.9e+06 3.9e+06 ... 3.89e+06 3.89e+06
  * bands        (bands) <U7 'B02_10m' 'B03_10m' ... 'B11_60m' 'B12_60m'
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

We now have a cube ready for work!